**Import Necessary Libraries**

In [1]:
#cv2
import cv2
import matplotlib.pyplot as plt
import numpy as np
#cvzone
from cvzone.HandTrackingModule import HandDetector

General Variables

In [2]:
maxHandCount= 2

#Colors for ease of use. Cv2 is BGR formatted
black=(0,0,0)
blue = (255,0,0)
green = (0,255,0)
red= (0,0,255)
white=(255,255,255)

font = cv2.FONT_HERSHEY_SIMPLEX


Library Variables

In [3]:
# Initialize Capture
cap = cv2.VideoCapture(1)

#Define hand detector
detector = HandDetector(maxHands=maxHandCount)

2024-06-03 18:40:54.827 Python[7075:457567] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1717429256.014147  457567 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Defined Functions

In [4]:
def drawHandNames(img,left,right):
    centerLeft= left['center']
    centerRight= right['center']
    cv2.putText(img, left["type"], centerLeft, font, 1, red, 2)
    cv2.putText(img, right["type"], centerRight, font, 1, blue, 2)

def minMaxControl(value,max,min=0):
    if value <= min:
        return min
    elif value >= max:
        return max
    else:
        return value

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


**Determine Hand Type Functionality**
Algoritma kameraya giren elleri farklı algılayabiliyor. Koordinatların doğru tespiti için net bir şekilde sağ ve sol eli bilmem gerekiyor.

In [5]:
def determineHandType(hand1,hand2):
    """
    Returns left and right hand clearly.

    Args:
        hand1 (dict): hand1 that algorithm caught.
        hand2 (dict): hand2 that algorithm caught.

    Returns:
        dict,dict: left,right hands.
    """
    if hand1["type"]=="Left":
        left= hand1
        right= hand2
    else:
        left= hand2
        right= hand1
    return left,right

W0000 00:00:1717429256.033126  457746 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1717429256.041148  457749 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


**Check State Functionality**
    Sol ve Sağ elin birbirine göre koordinatlarının farklı durumlarını 
hesaplamamı sağlıyor

In [6]:
def checkState(left,right):
    """
    Returns right hands relative location to left hand.

    Args:
        left (dict): x,y of Left Hand.
        right (dict): x,y of Right Hand.

    Returns:
        string: Relative Location.
    """
    if (right["x"]>left["x"]):
        #Right 3 possibility
        return "Right"+ checkStateY(left,right)
    elif (right["x"]<left["x"]):
        #Left 3 possibility
        return "Left"+ checkStateY(left,right)
    else:
        #Mid 3 possibility
        return "Mid"+ checkStateY(left,right)

def checkStateY(left,right):
    """
    Returns right hands relative Y(vertical) location to left hand.
    
    Args:
        left (dict): x,y of Left Hand.
        right (dict): x,y of Right Hand.

    Returns:
        string: Relative Y Location.
    """
    if (right["y"]<left["y"]):
        return "Up"
    elif (right["y"]>left["y"]):
        return "Down"
    else:
        return "Mid"
    

Core Loop

In [7]:
while True:
    # Kameradan veri oku
    success, img = cap.read()
    # Break if capture fails
    if not success:
        print("Couldn't read any data from capture")
        break


    maxVertical = img.shape[0]
    maxHorizontal = img.shape[1]

    # Görüntüde elleri tespit et
    hands, img = detector.findHands(img)

    if len(hands)==2:
        
        cv2.putText(img, f"{maxVertical,maxHorizontal}", (10, 50), font, 1, green, 2)

        #Determine the Hand Types Clearly
        left,right = determineHandType(hands[1],hands[0])


        x, y, w, h = left["bbox"]
        leftCoordinates= {"x":x,"y":y,"w":w,"h":h}
        x, y, w, h = right["bbox"]
        rightCoordinates= {"x":x,"y":y,"w":w,"h":h}
        # Adjust coordinates to ensure they are non-negative


        TopLeftCoordinates = {"x":minMaxControl(value=rightCoordinates["x"],max=maxHorizontal),
                              "y":minMaxControl(value=rightCoordinates["y"],max=maxVertical)}
        
        BottomRightCoordinates = {"x":minMaxControl(value=leftCoordinates["x"]+leftCoordinates["w"],max=maxHorizontal),
                                  "y":minMaxControl(value=leftCoordinates["y"]+leftCoordinates["h"], max=maxVertical)}
        

        state=checkState(leftCoordinates,rightCoordinates)
        cv2.putText(img,f"{state}",(rightCoordinates["x"],rightCoordinates["y"]),font,1,green,2)


        ####
        ####
        ####
        ####
        #Sol elin sol üst koordinatını 
        #Sağ elin sağ alt koordinatını almam gerekiyor
        #
        #Daha dorusu kesit alacağım fotoğrafın en sol üst koordinatını 
        #ve en sağ alt koordinatını doğru hesaplamam gerekiyor
        #
        #Bu hesapta 
        #Sol ve Sağ elin birbirine göre koordinatlarının farklı durumlarınnı 
        #hesaplamamı gerektiriyor
        ####
        ####
        ####
        ####

        #leftTop = min(0,yl)
        #rightBottom = min(yr + hr, img.shape[0])
        #leftLeft = min(0,xl)
        #rightRight = min(xr + wr, img.shape[1]) 

        #cv2.putText(img,f"{leftCoordinates} coords",left["center"],font,1,green,2)
        #cv2.putText(img,f"{rightCoordinates} coords",right["center"],font,1,blue,2)
    else:
        croppedIMG=img




    # Görüntüyü göster
    cv2.imshow("Camera", img)

    #cv2.imshow("Cropped",croppedIMG)

    # Wait for key press for a 1 ms
    key = cv2.waitKey(1)
    
    # If '0' key pressed then break
    if key == ord('0'):
        break

# Release all cameras and destroy openCV windows.
cap.release()
cap.release()
plt.ioff()  # Turn off interactive mode
plt.show()

2024-06-03 18:40:56.194 Python[7075:457567] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

        cv2.putText(img,f"{img.shape}",(20,50),font,1,white,2)
        cv2.putText(img,f"{xl},{yl},{wl},{hl} coords",left["center"],font,1,green,2)
        cv2.putText(img,f"{xr},{yr},{wr},{hr} coords",right["center"],font,1,blue,2)

(720,1280,3)

        if (TopLeftCoordinates["x"]> BottomRightCoordinates["x"] and
            TopLeftCoordinates["y"]> BottomRightCoordinates["y"]):
            #Change Coordinates
            TopLeftCoordinates["x"] = TopLeftCoordinates["x"] + rightCoordinates["w"]
            TopLeftCoordinates["y"] = TopLeftCoordinates["y"] + rightCoordinates["h"]

            BottomRightCoordinates["x"] = BottomRightCoordinates["x"] - leftCoordinates["w"]
            BottomRightCoordinates["y"] = BottomRightCoordinates["y"] - leftCoordinates["h"]

        croppedIMG = img[TopLeftCoordinates["y"]:BottomRightCoordinates["y"],
                         TopLeftCoordinates["x"]:BottomRightCoordinates["x"]]